<a href="https://colab.research.google.com/github/GeorgeCrossIV/AstraDB-Vector-Search-Yelp-Data/blob/main/Vector_Search_Yelp_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with this notebook

- Create a new vector search enabled database in Astra. [astra.datastax.com](https://astra.datastax.com)
- For the easy path, name the keyspace in that database "yelp" (otherwise be prepared to modify the CQL in this notebook)
- Create a token with permissions to create tables
- Download your secure-connect-bundle zip file.
- Download the Yelp dataset
- When you open this notebook in Google Colab or your own notebook server, drag-and-drop the secure connect bundle and yelp_academic_dataset_review.json into the File Browser of the notebook
- Set up a Hugging Face account and generate a token
- Update the Keys & Environment Variables cell in the notebook with information from the token you generated and the name of your secure connect bundle file.

# Setup

In [ ]:
!pip install pandas jupyter-datatables cassandra-driver transformers torch

# Imports

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
from transformers import AutoTokenizer, AutoModel, tokenization_utils
import numpy
import pandas as pd
import torch

# Keys & Environment Variables

In [ ]:
# keys and tokens here
cass_user = '<Astra DB ClientId goes here>'  # clientId
cass_pw = '<Astra DB Secret goes here>' # secret
scb_path = '/content/secure-connect-cassio-db.zip'
token = "<Hugging Face Token goes here>" # hugging face token
review_data_file = "yelp_academic_dataset_review-short.json"
model='sentence-transformers/all-MiniLM-L6-v2'

create_embeddings=False

# set keyspace
keyspace="yelp"    # Be sure to have created the yelp keyspace in Astra
tablename="review" # The yelp.review table will be created below

#Load the sample Yelp review data JSON

In [ ]:
!wget "https://raw.githubusercontent.com/GeorgeCrossIV/AstraDB-Vector-Search-Yelp-Data/main/yelp_academic_dataset_review-short.json"

# Select a model to compute embeddings

In [ ]:
# Function to get embedding from a model and tokenizer
def get_embedding(text, model, tokenizer):
    # Tokenize input text
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Get model output
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings from the last hidden state
    # You might also consider using pooled output for sentence-level embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)

    # Convert the tensor embeddings into a flat list of floats
    float_embeddings = embeddings.numpy().flatten().tolist()

    return float_embeddings

# Load pretrained MiniLM model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model, token=token)
model = AutoModel.from_pretrained(model, token=token)

# Test text
#text = "Create embeddings using MiniLM."

# Get embedding
#embedding = get_embedding(text, model, tokenizer)

# Print embedding
#print(embedding)

# Connect to the Cluster

In [ ]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.set_keyspace('yelp')
session

# Drop / Create Schema

In [ ]:
# only use this to reset the schema
if create_embeddings:
  session.execute(f"""DROP INDEX IF EXISTS {keyspace}.minilm_desc""")
  session.execute(f"""DROP TABLE IF EXISTS {keyspace}.{tablename}""")

In [ ]:
# # Create Table
if create_embeddings:
  session.execute(f"""
    CREATE TABLE {keyspace}.{tablename} (
        review_id Text PRIMARY KEY,
        user_id Text,
        business_id Text,
        stars INT,
        date TIMESTAMP,
        text TEXT,
        useful INT,
        funny INT,
        cool INT,
        minilm_description_embedding vector<float, 384>,
    )
  """)

  # # Create Index
  # Valid values for the similarity_function are COSINE (default), DOT_PRODUCT, or EUCLIDEAN
  session.execute(f"""
  CREATE CUSTOM INDEX IF NOT EXISTS minilm_desc
  ON {keyspace}.{tablename} (minilm_description_embedding) USING 'StorageAttachedIndex'
  WITH OPTIONS = {{ 'similarity_function': 'COSINE' }}
  """)


# Load the table with data and create text embeddings

In [ ]:
def load_yelp_review_data(file_path, number_of_rows=0):
    """
    Load Yelp data from a JSON file into a pandas DataFrame.

    Parameters:
    - file_path: str, the path to the JSON file.
    - number_of_rows: int, optional, the number of rows to load.
                      Load all rows if number_of_rows is missing or zero.

    Returns:
    - df: pandas DataFrame containing the loaded data.
    """
    try:
        if number_of_rows > 0:
            # Load a specified number of rows
            df = pd.read_json(file_path, lines=True, nrows=number_of_rows)
        else:
            # Load all rows if number_of_rows is missing or zero
            df = pd.read_json(file_path, lines=True)

        # Display the loaded DataFrame
        #print(df.head())

        return df

    except FileNotFoundError:
        print(f"Error: The file at path '{file_path}' does not exist.")
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")

# Load the data: For testing purposes, only five records are loaded. Change to 0 to load all of the data
review_data = load_yelp_review_data(review_data_file, 10)
review_data

In [ ]:
if create_embeddings:
  for id, row in review_data.iterrows():
    # Create Embedding for each review row, save them to the database
    embedding = get_embedding(row.text, model, tokenizer)
    query = SimpleStatement(
                f"""
                INSERT INTO {keyspace}.{tablename}
                (review_id, business_id, cool, funny, minilm_description_embedding, stars, text, useful, user_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
            )
    #display(row.text)

    session.execute(query, (row.review_id, row.business_id, row.cool, row.funny, embedding, row.stars, row.text, row.useful, row.user_id))




---


# Start using the index

In the steps up to this point, we have been creating a schema and loading the table with data, including embeddings we generated through the MiniLM Embedding API.

# Convert a query string into a text embedding to use as part of the query

This is where the real fun starts.  Provide a question or request to be used as the query.  The source sample database is mostly consumer electronics and appliances, so imagine you're talking to a customer service rep at Best Buy or another electronics store.

Here we use the same API that we used to calculate embeddings for each row in the database, but this time we are using your input question to calculate a vector to use in a query.

In [ ]:
customer_input = "Which review mention cycling?'"

embedding = get_embedding(customer_input, model, tokenizer)
#display(embedding)

Let's take a look at what a query against a vector index could look like.  The query vector has the same dimensions (number of entries in the list) as the embeddings we generated a few steps ago for each row in the database.

use the following similarity functions: similarity_dot_product, similarity_cosine, similarity_euclidean

documentation - https://docs.datastax.com/en/astra-serverless/docs/vector-search/cql.html#_calculate_the_similarity.

In [ ]:
query = SimpleStatement(
    f"""
    SELECT review_id, stars, text, similarity_cosine(minilm_description_embedding, {embedding}) as similarity
    FROM {keyspace}.{tablename}
    ORDER BY minilm_description_embedding ANN OF {embedding} LIMIT 5;
    """
    )
#display(query)

# Find the top 5 results using ANN Similarity

In [ ]:
results = session.execute(query)
top_reviews = results._current_rows

for row in top_reviews:
  print(f"""{row.similarity}, {row.review_id}, {row.stars}, {row.text}\n""")


Let's ask another question: What review mentions poultry?

In [ ]:
customer_input = "Which review mention poultry?'"
embedding = get_embedding(customer_input, model, tokenizer)

query = SimpleStatement(
    f"""
    SELECT review_id, stars, text, similarity_cosine(minilm_description_embedding, {embedding}) as similarity
    FROM {keyspace}.{tablename}
    ORDER BY minilm_description_embedding ANN OF {embedding} LIMIT 5;
    """
    )

results = session.execute(query)
top_reviews = results._current_rows

for row in top_reviews:
  print(f"""{row.similarity}, {row.review_id}, {row.stars}, {row.text}\n""")